This notebook is a basic tutorial that demonstrates how to configure a simulation using Concordia.

<a href="https://colab.research.google.com/github/google-deepmind/concordia/blob/main/examples/alice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
# @title Imports

from json import load
import numpy as np
from IPython import display

import sentence_transformers

from concordia.language_model.openrouter_model import OpenRouterLanguageModel

# Use it like any other Concordia language model
from concordia.language_model.openrouter_model import OpenRouterLanguageModel

from concordia.language_model import no_language_model

from concordia.prefabs.simulation import generic as simulation

import concordia.prefabs.entity as entity_prefabs
import concordia.prefabs.game_master as game_master_prefabs

from concordia.typing import prefab as prefab_lib
from concordia.utils import helper_functions

from dotenv import load_dotenv
load_dotenv()

True

In [80]:
# @title Language Model - pick your model and provide keys or select DISABLE_LANGUAGE_MODEL

# By default this colab uses GPT-4, so you must provide an API key.
# Note that it is also possible to use local models or other API models,
# simply replace this cell with the correct initialization for the model
# you want to use.
import os
import time
from concordia.language_model.openrouter_model import OpenRouterLanguageModel
from concordia.language_model.retry_wrapper import RetryLanguageModel
import openai  # If using OpenAI's error classes; adjust if using another provider

_API_KEY = os.environ.get('OPENROUTER_API_KEY')
MODEL_NAME = os.environ.get('MODEL_NAME', 'mistralai/mistral-small-3.1-24b-instruct:free')

if not _API_KEY:
    raise ValueError('OPENROUTER_API_KEY is required.')

# To debug without spending money on API calls, set DISABLE_LANGUAGE_MODEL=True
DISABLE_LANGUAGE_MODEL = False

# Enable verbose logging to see which entity is making each LLM call
VERBOSE_LOGGING = True

if not DISABLE_LANGUAGE_MODEL:
    base_model = OpenRouterLanguageModel(
        api_key=_API_KEY,
        model_name=MODEL_NAME
    )
    # Enable verbose logging if supported
    if hasattr(base_model, "verbose_logging"):
        base_model.verbose_logging = True

    # Define a custom retry wrapper to handle 429 errors with a 61s wait
    class CustomRetryLanguageModel(RetryLanguageModel):
        def __init__(self, model):
            super().__init__(
                model,
                retry_on_exceptions=(Exception,),
                retry_tries=10,
                retry_delay=61,  # Wait 61 seconds on rate limit
            )

        def sample_text(self, *args, **kwargs):
            try:
                return super().sample_text(*args, **kwargs)
            except openai.error.RateLimitError as e:
                print("Rate limit hit. Waiting 61 seconds before retrying...")
                time.sleep(61)
                return super().sample_text(*args, **kwargs)

        def sample_choice(self, *args, **kwargs):
            try:
                return super().sample_choice(*args, **kwargs)
            except openai.error.RateLimitError as e:
                print("Rate limit hit. Waiting 61 seconds before retrying...")
                time.sleep(61)
                return super().sample_choice(*args, **kwargs)

    model = CustomRetryLanguageModel(base_model)
else:
    from concordia.language_model import no_language_model
    model = no_language_model.NoLanguageModel()


In [81]:
# @title Setup sentence encoder

if DISABLE_LANGUAGE_MODEL:
  embedder = lambda _: np.ones(3)
else:
  st_model = sentence_transformers.SentenceTransformer(
      'sentence-transformers/all-mpnet-base-v2')
  embedder = lambda x: st_model.encode(x, show_progress_bar=False)

In [83]:
test = model.sample_text(
    'Is societal and technological progress like getting a clearer picture of '
    'something true and deep?')
print(test)

The analogy of societal and technological progress as getting a clearer picture of something true and deep is an interesting one and has both strengths and limitations. Let's explore this idea from a few angles:

### Strengths of the Analogy:

1. **Increasing Understanding**: Just as a clearer picture reveals more details, societal and technological progress often involves gaining deeper insights into various aspects of life, such as science, ethics, and human behavior.

2. **Iterative Improvement**: Both getting a clearer picture and making societal progress often involve iterative processes. Each new discovery or technological advancement can refine our understanding and improve our capabilities.

3. **Complexity and Depth**: The idea that there is something inherently true and deep to be uncovered aligns with the notion that progress reveals underlying principles and truths about the world and human experience.

### Limitations of the Analogy:

1. **Subjectivity and Bias**: Unlike a

In [84]:
# @title Load prefabs from packages to make the specific palette to use here.

prefabs = {
    **helper_functions.get_package_classes(entity_prefabs),
    **helper_functions.get_package_classes(game_master_prefabs),
}

In [ ]:
#@title Print menu of prefabs

display.display(
    display.Markdown(helper_functions.print_pretty_prefabs(prefabs)))

In [85]:
# @title Configure instances.

instances = [
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': 'Alice',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='basic__Entity',
        role=prefab_lib.Role.ENTITY,
        params={
            'name': 'White Rabbit',
            'goal': 'Get to the queen on time',
        },
    ),
    prefab_lib.InstanceConfig(
        prefab='generic__GameMaster',
        role=prefab_lib.Role.GAME_MASTER,
        params={
            'name': 'default rules',
            'acting_order': 'fixed',
        },
    ),
]

In [87]:
config = prefab_lib.Config(
    default_premise=(
        'Alice is sitting on the bank, being quite bored and tired by reading a'
        ' book with no pictures or conversations, when suddenly a white rabbit'
        ' with pink eyes ran close by her. '
    ),
    default_max_steps=2,
    prefabs=prefabs,
    instances=instances,
)


# The simulation

In [88]:
# @title Initialize the simulation
runnable_simulation = simulation.Simulation(
    config=config,
    model=model,
    embedder=embedder,
)

In [ ]:
# @title Multiple Models Example (Optional)
# This shows how to use different models for different entities

# Uncomment to try different models for different entities:

# from concordia.language_model.lmstudio_model import LMStudioLanguageModel

# # Create different models with verbose logging
# openrouter_model = OpenRouterLanguageModel(
#     api_key=_API_KEY, 
#     model_name=MODEL_NAME,
#     verbose_logging=True  # Enable detailed logging
# )

# lmstudio_model = LMStudioLanguageModel(
#     model_name="local-llama",
#     base_url="http://localhost:1234/v1",
#     verbose_logging=True  # Enable detailed logging
# )

# # To assign different models to entities, you would modify the simulation
# # initialization to pass model-specific configurations

In [89]:
# @title Run the simulation
results_log = runnable_simulation.play()

Terminate? No
Only one game master available (default rules), skipping the call to `next_game_master`.
Entity Alice observed: //Present Time// Alice observes the White Rabbit pausing briefly, looking around nervously, and then pulling out a small pocket watch from its waistcoat. It glances at the watch, mutters something to itself, and then hurriedly continues running, disappearing into the underbrush. The sound of its hurried footsteps fades into the distance, leaving Alice alone on the bank with her book.
Entity White Rabbit observed: ```
//The present// As White Rabbit dashes through the underbrush, it hears the rustling of leaves and the distant murmur of a river. The sun filters through the canopy above, casting dappled shadows on the forest floor. The scent of wildflowers fills the air, and the sound of Alice's book closing echoes faintly behind it. The rabbit's heart races as it continues to flee, unaware of what might be pursuing it.
```
Entity White Rabbit is next to act. They

In [90]:
# @title Display the log
display.HTML(results_log)